# Visualitzacions incentius a l'autoconsum

Repo at github [ctrl-alt-d/viz_autoconsum](https://github.com/ctrl-alt-d/viz_autoconsum)

Aquest codi es descarrega el PDF d'incentius a l'autoconsum, extreu el text, l'agrega i el mostra en forma de mapa de calor i de barres.


## Descarrega de fitxers

In [90]:
import wget
fitxers = [
    "https://icaen.gencat.cat/web/.content/20_Energia/26_ajuts_financament/2021_IDAE_EERR/Arxius/Resolucio_Atorgament_PUBLICABLE-st_ah_v3_L1_1.pdf",
    # Falten 3 fitxers més
]

url = fitxers[0]
file_path = wget.download(url, './data/f1.pdf')

## Extracció de les dades

Estreure les dades del PDF i convertir-les a data frame

### Extreure el text del pdf

In [91]:
import PyPDF2

# creating an object 
file = open(file_path, 'rb')

# creating a pdf reader object
reader = PyPDF2.PdfFileReader(file)

s = ""
for page in list(reader.pages):
    s += page.extract_text()

from_char = s.find("Codi expedient ")
data = s[from_char:].replace("\n", "").split(" ICA")[1:]

### Passar el text a llista

In [92]:
def tokenize(item):
    """025/21/000332  ***0530**  GESTINGRAL SL  Instal.lació de generació de 9,57 kW  Vilanova i la Geltrú  11369,16  5.116,12  0,00  15/12/2021 09:25:44 
    098765432109876543210987654321098765432109876543210987654321098765432109876543210987654321098765432109876543210987654321098765432109876543210987654321
    """

    if item[15:18] != "***":
        return None

    # normalitzar espais
    item0 = item.replace("  ", " ")

    # data: al final de tot
    last_second = item0.rfind(":")
    item1 = item0[:last_second + 3]
    dia = item1[-19:]

    # inversió: primer dels tres darrers imports
    item2 = item1[:-22]
    items = item2.split(" ")
    inversio_justificar = float( items[-3].replace(".", "").replace(",", ".") )

    # poblacio: abans dels tres darrers imports i després del darrer kW
    items = items[:-3]
    item3 = " ".join(items)
    from_kWh = item3.rfind("kW") + 3
    poblacio = item3[from_kWh:].replace(" -", "-") # i treure espai davant guió.
    poblacio_split = poblacio.split(", ")
    poblacio_split = poblacio_split[-1:] + poblacio_split[:-1]
    poblacio = " ".join(poblacio_split).strip().replace("  ", " ").replace("' ", "'")   
    poblacio = poblacio.replace("Capmany", "Campmany") 

    # kw: entre els dos textos
    from_generacio = item0.find("generació de ") + len("generació de ")
    to_kw = item0.find(" kW")
    kW = float(item0[from_generacio:to_kw].replace(",","."))

    return {
        "dia": dia,
        "inversio_justificar": inversio_justificar,
        "kW": kW,
        "poblacio": poblacio,
        "linia": item
        }

recordset = [tokenize(d) for d in data]


### Coordenades de Poblacions wikipedia

Agafo les coordenades de tots els municipis de Catalunya de la WikiData.

Gràcies [@miquelduran](https://twitter.com/miquelduran) i [@paucabot](https://twitter.com/paucabot) pel cop de mà amb la consulta Sparql.

In [93]:
from time import sleep
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import json
import os

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
_cache = "./data/poblacions_wikidata.json"

def getlatlong(txt):
    txt = txt.replace("Point(", "").replace(")", "")
    p = txt.split(" ")
    return (p[1], p[0])

def getcoordenades_wikipedia():
    sparql.setQuery("""
        # @miquelduran & @paucabot credits:
        SELECT ?muni ?muniLabel ?coord WHERE {
        ?muni wdt:P31 wd:Q33146843;
        wdt:P625 ?coord.        
        SERVICE wikibase:label { bd:serviceParam wikibase:language "ca" . }
        }
        ORDER BY ?place
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.json_normalize(results['results']['bindings'])
    return {
        k: getlatlong(v)
        for k, v in
        results_df[['muniLabel.value', 'coord.value']].values
    }

def getcoordenades_all():    
    if not os.path.isfile(_cache):
        coor = getcoordenades_wikipedia()
        setcoordinates_cache(coor)

    with open(_cache,) as infile:
        return json.load(infile)

def setcoordinates_cache(poblacio_wikidata):
    with open(_cache, 'w') as outfile:
        json.dump(poblacio_wikidata, outfile, indent=2)

def getcoordenades(poblacio):
    "Donada una població, retornem la seva lat i long"
    coors = getcoordenades_all()
    r = next( v for k, v in coors.items() if k.lower() == poblacio.lower())
    return r



Afegim lat i long a cada instal·lació (a la població de la instal·lació)

In [94]:
for l in recordset:
    coor = getcoordenades(l['poblacio'])
    l["lat"] = float(coor[0])
    l["long"] = float(coor[1])

### Agreguem les dades

In [95]:
import pandas as pd
df = pd.DataFrame(recordset)
data_agg = (
    df
    .groupby(['poblacio', 'lat', 'long'], as_index=False)
    .agg({'dia':'count', 'kW': 'sum'})
    .rename(columns={'dia':'count'})
    )



### Visualització al Mapa

In [96]:
from folium.plugins import HeatMap
import folium
from branca.element import Figure
from itertools import groupby

# crear el map
mapObj = folium.Map(location=getcoordenades('Santpedor'), zoom_start = 8, width=750, height=500)

# Afegir capa de color
data = [tuple(r) for r in data_agg[["lat", "long", "kW"]].to_numpy()]
heatmap = HeatMap( data,
                   min_opacity=0.2,
                   radius=50, blur=50, 
                   max_zoom=1)
heatmap.add_to(mapObj)

# afegim marques
for r in data_agg.to_dict('records'):
    depot_node = (r["lat"], r["long"])
    poblacio = r["poblacio"]
    n_instalacions = r["count"]
    kW = r["kW"]
    tooltip = f"{poblacio}: {n_instalacions} instal·lacions. {kW} kW"
    folium.CircleMarker(location=depot_node,
                        radius=2,    
                        fill_color ='green',
                        fill_opacity=0.2,
                        tooltip=tooltip
                        ).add_to(mapObj)    

# pintar
fig = Figure(width=750, height=500)
fig.add_child(mapObj)

### Visualització barres

In [97]:
import altair as alt
import pandas as pd

df = pd.DataFrame(recordset)

(
    alt
    .Chart(df)
    .mark_bar()
    .encode(
        alt.X("sum(kW):Q",title="Inversió a justificar", stack='zero'),
        alt.Y("poblacio:N", sort="-x", title=""),
        tooltip=["count(kW):Q", "sum(kW):Q"]
    )
    .interactive()
)


alt.Chart(...)